In [21]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle
import pandas as pd

In [2]:
pred_data = np.load(r'results\stock_000001SZustd_50_1_PatchTST_stock_custom_ftMS_sl50_ll25_pl1_dm128_nh8_el3_dl1_df256_fc1_ebtimeF_dtTrue_Exp_0\pred.npy')
print(pred_data.shape)
pred_data = pred_data.reshape((pred_data.shape[0]))
print(pred_data[:15])

true_data = np.load(r'results\stock_000001SZustd_50_1_PatchTST_stock_custom_ftMS_sl50_ll25_pl1_dm128_nh8_el3_dl1_df256_fc1_ebtimeF_dtTrue_Exp_0\true.npy')
true_data = true_data.reshape((true_data.shape[0]))
print(true_data[:15])

(800, 1, 1)
[-0.24318472 -0.10326846  0.14551774  0.09388182  0.16411474  0.09579294
  0.08957873 -0.23152643 -0.16371813  0.00643662  0.04437798  0.08180664
  0.16832884  0.50132424  0.14367586]
[-0.28807533  0.2936211   0.84149134  0.04105194  0.12143223  0.17425466
 -0.6780516  -0.12085529 -1.0155877  -1.124596    0.01754558  0.44627866
  2.3314888   0.33241615  1.5396876 ]


In [3]:
#load scaler
with open(r'scaler\stock_000001SZustd\stock_000001SZustd_50_1.pkl', 'rb') as f:
    scaler = pickle.load(f)

c:\Users\Vincent\.conda\envs\Pytorch_1_11\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
tc =  np.load('scaler\stock_000001SZustd\stock_000001SZustd_50_1.npy')
print(tc.shape)

(2, 10)


In [5]:
print(pred_data.shape)
print(pred_data[:10])

(800,)
[-0.24318472 -0.10326846  0.14551774  0.09388182  0.16411474  0.09579294
  0.08957873 -0.23152643 -0.16371813  0.00643662]


In [6]:
pred_tmp_t = np.full((pred_data.shape[0], 9), 0, np.float32)
print(pred_tmp_t.shape)
pred_tmp_t = np.insert(pred_tmp_t, 9, pred_data, axis=1)
print(pred_tmp_t.shape)


(800, 9)
(800, 10)


In [7]:
#get the orginal 
pred_orig = scaler.inverse_transform(pred_tmp_t)[:,-1]
print(pred_orig[:10])

[-0.5323521  -0.16628733  0.48461524  0.3495195   0.53327084  0.3545196
  0.33826125 -0.50185037 -0.32444263  0.12073547]


In [8]:
def data_inverse(label_c, original_dimension, original_position, scaler): #original dimensino(2D) without the label_c column
    temp_matrix = np.full(original_dimension, 0, np.float32)
    temp_matrix = np.insert(temp_matrix, original_position, label_c, axis=1)
    return scaler.inverse_transform(temp_matrix)[:, original_position]

In [9]:
# print(pred_orig[:10] == data_inverse(pred_data, (pred_data.shape[0], 9), 9, scaler)[:10])

In [10]:
true_y = data_inverse(true_data, (true_data.shape[0], 9), 9, scaler)
print(true_y.shape)

A_x = np.genfromtxt('PatchTST_supervised\stock_data\stock_000001SZustd.csv', delimiter=',', skip_header=1)
A_x = A_x.astype(np.float32)
A_x = A_x[: , 1:]
print(A_x[:5])
print(A_x.shape)
print(A_x.dtype)
print(A_x[:,0].shape)

(800,)
[[ 3.6099999e+00  1.0841413e+05  6.7332305e+04 -2.2471910e+00
   1.4354067e+00 -1.6722408e-01  4.2919999e-01  4.2918077e-01
  -3.1142139e+00 -2.5605137e+00]
 [-1.9000000e+00  1.0686837e+05  6.6813688e+04  3.1198688e+00
   6.2893081e-01  2.5125628e+00 -1.2294000e+00 -1.2293609e+00
   6.9516301e-01  4.1986537e-01]
 [ 0.0000000e+00  7.9396477e+04  4.9387039e+04 -1.5923567e+00
  -9.3750000e-01  3.2679740e-01 -1.7776000e+00 -1.7775663e+00
  -1.3017663e+00 -1.8708645e+00]
 [-2.7500000e+00  1.1148138e+05  6.7292188e+04  1.6181229e-01
  -1.5772871e+00 -3.9087949e+00  1.9450000e-01  1.9450904e-01
  -1.8292873e+00 -1.7309900e-01]
 [-1.9900000e+00  5.4461621e+04  3.2003586e+04 -2.7463651e+00
  -3.5256410e+00 -2.3728814e+00 -1.3827000e+00 -1.3826935e+00
   2.1324158e-01 -1.4597582e+00]]
(4139, 10)
float32
(4139,)


In [18]:
def find_subset_pos(A, B):
    print(A.shape)
    print(B.shape)
    for i in range(A.shape[0] - B.shape[0] + 1):
        if np.array_equal(A[i:i+B.shape[0]], B):
            return i
    return -1

In [19]:
find_subset_pos(A_x[:, 1].astype(np.int16), true_y.astype(np.int16))

(4139,)
(800,)


-1

In [16]:
find_subset_pos(np.array([1,2,3,4]), np.array([4.2]))


-1

In [55]:
#pd date time

s1 = pd.read_csv(r'PatchTST_supervised\stock_data\stock_000001SZustd.csv')
format_string = '%Y%m%d'

In [56]:
ts1 = pd.to_datetime(s1.date, format='%Y%m%d')

In [57]:
ts3 = pd.to_datetime(s1.date, format=format_string)

In [59]:
ts3

0      2005-05-10
1      2005-05-11
2      2005-05-12
3      2005-05-13
4      2005-05-16
          ...    
4134   2022-12-09
4135   2022-12-12
4136   2022-12-13
4137   2022-12-14
4138   2022-12-15
Name: date, Length: 4139, dtype: datetime64[ns]

In [60]:
ts1

0      2005-05-10
1      2005-05-11
2      2005-05-12
3      2005-05-13
4      2005-05-16
          ...    
4134   2022-12-09
4135   2022-12-12
4136   2022-12-13
4137   2022-12-14
4138   2022-12-15
Name: date, Length: 4139, dtype: datetime64[ns]

In [44]:
ts2 = pd.to_datetime(ts1)

In [47]:
ts2.values

array(['2005-05-10T00:00:00.000000000', '2005-05-11T00:00:00.000000000',
       '2005-05-12T00:00:00.000000000', ...,
       '2022-12-13T00:00:00.000000000', '2022-12-14T00:00:00.000000000',
       '2022-12-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [61]:
ts2

0      2005-05-10
1      2005-05-11
2      2005-05-12
3      2005-05-13
4      2005-05-16
          ...    
4134   2022-12-09
4135   2022-12-12
4136   2022-12-13
4137   2022-12-14
4138   2022-12-15
Name: date, Length: 4139, dtype: datetime64[ns]

In [53]:
pd.tseries.frequencies.to_offset('2t')

<2 * Minutes>